In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

# This notebook is for investigating how one can meaningfully<br> 1) sort the files into observation types,<br> 2) group/select the files for a given observation type on GOATS' dragons page.<br> 

Implementing a generalized sorting mechanism is a necessity for GOATS application. This will ensure users can properly manage observations, especially when there are, say data taken with multiple filters or multiple science targets within the same observation ID.    

In [2]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import inspect

# for bokeh apps to load from the jupyter notebook on a different browser tab
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments

from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

from recipe_system.mappers.primitiveMapper import PrimitiveMapper
from recipe_system.mappers.recipeMapper import RecipeMapper

import textwrap

In [4]:
from recipe_system.reduction.coreReduce import Reduce
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

## Modifying the filelist generation to be generic

In [5]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    obs_types = ['OBJECT','BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','CAL','FRINGE','MOS MASK', 'BPM'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    all_meta = {}

    # Note that even if a few keywords are extracted below, thiis is just meant for demonstration puporses. 
    # If onyl these keywords are extracted in production, it will not align with  
    # implementation of item (2), allowing users to group/select preferred header keywords
    for K in obs_types:
        all_meta[K] = {'file':[],
                        'obs_class':[],
                        'group_id':[],
                        'exp':[],
                        'object':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        if "BPM" in ad.tags or "UNPREPARED" in ad.tags: ## want only raw, i.e., "unprepared" files, but BPM is an exception, which is processed/prepared 
            K = ad.observation_type() ## astrodata header also has the observation type, which should match what's in the archive drop-down menu
        
        if "PREPARED" in ad.tags or "PROCESSED" in ad.tags: ## skip all "prepared"/"processed" files
            continue
            
        all_meta[K]['file'].append(F)
        all_meta[K]['obs_class'].append(ad.observation_class())
        
        # group_id seems to be not implemented for GNIRS spectroscopy yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['object'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


# GMOS longslit example

In [6]:
# data_path = "/data/goats_dev_data/example_data/data/ZTF18acppavh/GEM" ## linux machine 
# #data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats/goats_data/ZTF18aabfthf/GEM" ## mac 
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats" ## mac
obsid = 'GS-2021A-DD-102-9'

In [7]:
all_meta = generate_filelists(data_path, obsid)
observation_type = []

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}")
    observation_type.append(K)


The total number of files for observation ID GS-2021A-DD-102-9 is 59
There are 4 files for observation type OBJECT
There are 50 files for observation type BIAS
There are 2 files for observation type FLAT
There are 2 files for observation type ARC
There are 1 files for observation type BPM


## Let's create a separate pandas dataframe for each observation type

In [8]:
DF_object = pd.DataFrame(all_meta['OBJECT'])
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_arc = pd.DataFrame(all_meta['ARC'])
DF_bpm = pd.DataFrame(all_meta['BPM'])


## Let's examine the "unique" object names (not to be confused with the OBJECT observation type; this refers to the header keyword here) 

In [9]:
print (np.unique(DF_object['object'].values))

['AT2020caa' 'Twilight']


In [10]:
print (np.unique(DF_bias['object'].values))

['Bias']


In [11]:
print (np.unique(DF_flat['object'].values))

['GCALflat']


In [12]:
print (np.unique(DF_arc['object'].values))

['CuAr']


In [13]:
print (np.unique(DF_bpm['object'].values))

['BPM']


**Conclusion: we Do NOT expect observation types other than `OBJECT` to have different "object names" (again, referring to header keyword). Hence, only the `OBJECT` observation type should be separated based on the unique object names when listing the files on the GOATS frontend.** 

#

# About feature to "group"/"select" by keyword

## One possibility is to group the files for a given observation type using the descriptor `group_id` -- pretty ugly-looking string (see below). This descriptor so defined in the DRAGONS codebase is meant to group "compatible" observations for a given instrument, mode of observation, data type, etc. However, for some instrumet+mode combination, it may not yet exist, e.g., GNIRS spectroscopy.

In [10]:
for K in observation_type:
    print (f"\nobservation type {K}")
    DF = pd.DataFrame(all_meta[K])

    for kk, vv in DF.groupby('group_id'):
        print (f"Group ID is: {kk}")
        print (f"No. of files for this group ID: {len(vv)}")



observation type BIAS
Group ID is: 2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1:4224]", "'BI5-36-4k-2, 3':[513:1024,1:4224]", "'BI5-36-4k-2, 2':[1025:1536,1:4224]", "'BI5-36-4k-2, 1':[1537:2048,1:4224]", "'BI11-33-4k-1, 4':[2049:2560,1:4224]", "'BI11-33-4k-1, 3':[2561:3072,1:4224]", "'BI11-33-4k-1, 2':[3073:3584,1:4224]", "'BI11-33-4k-1, 1':[3585:4096,1:4224]", "'BI12-34-4k-1, 4':[4097:4608,1:4224]", "'BI12-34-4k-1, 3':[4609:5120,1:4224]", "'BI12-34-4k-1, 2':[5121:5632,1:4224]", "'BI12-34-4k-1, 1':[5633:6144,1:4224]"]
No. of files for this group ID: 50

observation type FLAT
Group ID is: GG455_6.5e-07_2_2_Normal_["'BI5-36-4k-2, 4':[1:512,1:4224]", "'BI5-36-4k-2, 3':[513:1024,1:4224]", "'BI5-36-4k-2, 2':[1025:1536,1:4224]", "'BI5-36-4k-2, 1':[1537:2048,1:4224]", "'BI11-33-4k-1, 4':[2049:2560,1:4224]", "'BI11-33-4k-1, 3':[2561:3072,1:4224]", "'BI11-33-4k-1, 2':[3073:3584,1:4224]", "'BI11-33-4k-1, 1':[3585:4096,1:4224]", "'BI12-34-4k-1, 4':[4097:4608,1:4224]", "'BI12-34-4k-1, 3':[4609:5120,1:4

## F2 imaging example -- containing observations (same observation ID) taken with multiple filters 

In [11]:
data_path = "/data/goats_dev_data/example_data/data/ZTF18acppavh/GEM" ## linux machine 
obsid = 'GS-2015B-Q-46-59'

In [13]:
all_meta = generate_filelists(data_path, obsid)
observation_type = []

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}")
    observation_type.append(K)


The total number of files for observation ID GS-2015B-Q-46-59 is 235
There are 47 files for observation type DARK
There are 32 files for observation type FLAT
There are 6 files for observation type standard
There are 150 files for observation type object


In [14]:
for K in observation_type:
    print (f"\nobservation type {K}")
    DF = pd.DataFrame(all_meta[K])

    for kk, vv in DF.groupby('group_id'):
        print (f"Group ID is: {kk}")
        print (f"No. of files for this group ID: {len(vv)}")



observation type DARK
Group ID is: 13.0_1_1_['[1:2048,1:2048]']
No. of files for this group ID: 10
Group ID is: 15.0_1_1_['[1:2048,1:2048]']
No. of files for this group ID: 10
Group ID is: 2.0_1_1_['[1:2048,1:2048]']
No. of files for this group ID: 7
Group ID is: 3.0_1_1_['[1:2048,1:2048]']
No. of files for this group ID: 10
Group ID is: 8.0_1_1_['[1:2048,1:2048]']
No. of files for this group ID: 10

observation type FLAT
Group ID is: H_f/16_G5830_3.0_GS-2015B-Q-46-60_1_['[1:2048,1:2048]']_F2_IMAGE_FLAT
No. of files for this group ID: 10
Group ID is: H_f/16_G5830_3.0_GS-2015B-Q-46-62_1_['[1:2048,1:2048]']_F2_IMAGE_FLAT
No. of files for this group ID: 10
Group ID is: Ks_f/16_G5830_8.0_GS-2015B-Q-46-60_1_['[1:2048,1:2048]']_F2_IMAGE_FLAT
No. of files for this group ID: 6
Group ID is: Ks_f/16_G5830_8.0_GS-2015B-Q-46-62_1_['[1:2048,1:2048]']_F2_IMAGE_FLAT
No. of files for this group ID: 6

observation type standard
Group ID is: GS-2015B-SV-301-46_H_f/16_G5830_3.0_1_['[1:2048,1:2048]']
No.

## What if the users wants a separate grouping? If we resort to using the `group_id` descriptor only, this will limit a more flexible grouping mechannism on GOATS. 

## We already display the `astrodata` generated descriptors (which tend to be self-explanatory as well) on the GOATS dragons page. Best way forward, offering the maximum flexibility, is to allow users to choose the descriptors to group the files themselves.   

**E.g., Let's try to choose a bunch of descriptors to group the flats for the above F2 imaging observation ID. I'm using pandas dataframe for this.**

In [15]:
df_flat = pd.DataFrame(all_meta['FLAT'])

In [17]:
AD = astrodata.open(df_flat['file'].values[0])

In [18]:
AD.descriptors

('airmass',
 'amp_read_area',
 'ao_seeing',
 'array_name',
 'array_section',
 'azimuth',
 'binning',
 'calibration_key',
 'cass_rotator_pa',
 'central_wavelength',
 'coadds',
 'data_label',
 'data_section',
 'dec',
 'decker',
 'detector_name',
 'detector_roi_setting',
 'detector_rois_requested',
 'detector_section',
 'detector_x_bin',
 'detector_x_offset',
 'detector_y_bin',
 'detector_y_offset',
 'disperser',
 'dispersion',
 'dispersion_axis',
 'effective_wavelength',
 'elevation',
 'exposure_time',
 'filter_name',
 'focal_plane_mask',
 'gain_setting',
 'gcal_lamp',
 'group_id',
 'instrument',
 'is_ao',
 'is_coadds_summed',
 'is_in_adu',
 'local_time',
 'lyot_stop',
 'mdf_row_id',
 'nominal_atmospheric_extinction',
 'nominal_photometric_zeropoint',
 'object',
 'observation_class',
 'observation_epoch',
 'observation_id',
 'observation_type',
 'overscan_section',
 'pixel_scale',
 'position_angle',
 'program_id',
 'pupil_mask',
 'qa_state',
 'ra',
 'raw_bg',
 'raw_cc',
 'raw_iq',
 'raw_

**Let's make a pandas dataframe containing all the descriptors for each flat file**

In [32]:
TEMP = {}
for i, K in enumerate(df_flat['file'].values):
    temp = {}
    temp['file'] = K
    ad = astrodata.open(K)
    for X in ad.descriptors:
        result = getattr(ad, X)
        temp[X] = result()
    TEMP[i] = temp  

In [33]:
descriptor_DF = pd.DataFrame.from_dict(TEMP).transpose()

In [34]:
descriptor_DF.head()

,file,airmass,amp_read_area,ao_seeing,array_name,array_section,azimuth,binning,calibration_key,cass_rotator_pa,...,telescope_x_offset,telescope_y_offset,ut_date,ut_datetime,ut_time,wavefront_sensor,wavelength_band,wcs_dec,wcs_ra,well_depth_setting
0,/data/goats_dev_data/example_data/data/ZTF18ac...,1.0,[None],None,[None],"[(0, 2048, 0, 2048)]",146.999983,1x1,GS-2015B-Q-46-60-001,175.10997,...,-0.0,-0.0,2015-11-28,2015-11-28 11:31:33.400000,11:31:33.400000,None,H,-30.144968,168.897162,None
1,/data/goats_dev_data/example_data/data/ZTF18ac...,1.0,[None],None,[None],"[(0, 2048, 0, 2048)]",146.999983,1x1,GS-2015B-Q-46-60-002,175.10997,...,-0.0,-0.0,2015-11-28,2015-11-28 11:31:56.900000,11:31:56.900000,None,H,-30.144843,168.995078,None
2,/data/goats_dev_data/example_data/data/ZTF18ac...,1.0,[None],None,[None],"[(0, 2048, 0, 2048)]",146.999983,1x1,GS-2015B-Q-46-60-003,175.10997,...,-0.0,-0.0,2015-11-28,2015-11-28 11:32:19.900000,11:32:19.900000,None,H,-30.144718,169.091012,None
3,/data/goats_dev_data/example_data/data/ZTF18ac...,1.0,[None],None,[None],"[(0, 2048, 0, 2048)]",146.999983,1x1,GS-2015B-Q-46-60-004,175.10997,...,-0.0,-0.0,2015-11-28,2015-11-28 11:32:42.400000,11:32:42.400000,None,H,-30.144595,169.183942,None
4,/data/goats_dev_data/example_data/data/ZTF18ac...,1.0,[None],None,[None],"[(0, 2048, 0, 2048)]",146.999983,1x1,GS-2015B-Q-46-60-005,175.10997,...,-0.0,-0.0,2015-11-28,2015-11-28 11:33:04.400000,11:33:04.400000,None,H,-30.144473,169.276857,None


**Suppose the user wants to group by binning and wavelength_band, i.e., make subgroups of files, which have the same values for the two parameters binning and wavelength_band**

In [39]:
for K, V in descriptor_DF.groupby(['binning','wavelength_band']):
    print (K, len(V))

('1x1', 'H') 20
('1x1', 'K') 12


## We can also consider allowing users to fine-tune their selection of files, beyond simply checking the button on and off. For that let's follow what IPAC has https://irsa.ipac.caltech.edu/onlinehelp/ztf/<br>We allow the following six logical operators:

-  = which means 'equal to' (exactly!), e.g., the parameter on which you are querying (the column headers as shown) is exactly equal to this value you are specifying.
-  \> which means 'greater than'
- < which mean 'less than'
- != which means 'not equal to' (exactly!)
- \>= which means 'greater than or equal to'
- <= which means 'less than or equal to'

In [40]:
descriptor_DF.columns

Index(['file', 'airmass', 'amp_read_area', 'ao_seeing', 'array_name',
       'array_section', 'azimuth', 'binning', 'calibration_key',
       'cass_rotator_pa', 'central_wavelength', 'coadds', 'data_label',
       'data_section', 'dec', 'decker', 'detector_name',
       'detector_roi_setting', 'detector_rois_requested', 'detector_section',
       'detector_x_bin', 'detector_x_offset', 'detector_y_bin',
       'detector_y_offset', 'disperser', 'dispersion', 'dispersion_axis',
       'effective_wavelength', 'elevation', 'exposure_time', 'filter_name',
       'focal_plane_mask', 'gain_setting', 'gcal_lamp', 'group_id',
       'instrument', 'is_ao', 'is_coadds_summed', 'is_in_adu', 'local_time',
       'lyot_stop', 'mdf_row_id', 'nominal_atmospheric_extinction',
       'nominal_photometric_zeropoint', 'object', 'observation_class',
       'observation_epoch', 'observation_id', 'observation_type',
       'overscan_section', 'pixel_scale', 'position_angle', 'program_id',
       'pupil_mask',

**Let's select flat files based on exposure_time**

In [45]:
mask = descriptor_DF['exposure_time']<5.0 ## exposure time less than 5 seconds
selected1 = descriptor_DF[mask]
print (f"Total number of selected files is {len(selected1)}")

Total number of selected files is 20
